In [1]:
# import
import zipfile


In [5]:
# extract zip file quickly and in gitignore'd folder

zip_file_path = "./data/archive.zip"

# Step 2: Extract the downloaded zip file
extracted_folder_path = "./data/RoadDetectionFiles"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [4]:
# Define paths

test_image = './data/RoadDetectionFiles/Test/image/'
train_image = './data/RoadDetectionFiles/Train/image/'
validation_image = './data/RoadDetectionFiles/Validation/image/'

test_centerline = './data/RoadDetectionFiles/Test/centerline/'
train_centerline = './data/RoadDetectionFiles/Train/centerline/'
validation_centerline = './data/RoadDetectionFiles/Validation/centerline/'

test_label = './data/RoadDetectionFiles/Test/label/'
train_label = './data/RoadDetectionFiles/Train/label/'
validation_label = './data/RoadDetectionFiles/Validation/label/'

options we are ditching
- find pretrained unet (too hard)
- train a unet for semantic segmentation (takes too much time)


option we are doing:
- resize label images to be all same size - same size as image
- make a disclaimer that 44 photos is very few
- calculate the sum score for each image - number of pixels classified as a road
- Train a CNN network to predict number of road pixels (similar to HW3 , but the output is a single score)


- extension: instead of using regular images, crop patches to like 100x100 images


In [5]:
# create function to crop all images to be the same size - since the images are all different aspect ratios, the images need to be cropped, not just resized

from PIL import Image
import os

def crop_to_square(image_path, output_path, dimension):
    # Open the image
    with Image.open(image_path) as img:
        
        # Calculate crop box coordinates
        left = (img.width - dimension) / 2
        top = (img.height - dimension) / 2
        right = (img.width + dimension) / 2
        bottom = (img.height + dimension) / 2

        # Crop the image to the square
        cropped_img = img.crop((left, top, right, bottom))

        # Save the cropped image
        cropped_img.save(output_path)



In [6]:
# find the largest square that can fit inside all the images - this will be the dimensions to crop the image

def find_crop_dim(folder):
    max_dimension = 10000 # placeholder
    for filename in os.listdir(folder):
        if filename.endswith('.bmp'):
            input_path = os.path.join(folder, filename)
            # img = Image.open(input_path)
            with Image.open(input_path) as img:
                max_dimension = min(max_dimension, min(img.width, img.height))
    return max_dimension

# the label images are extracted from the satellite images, so it makes sense they are the same dimension

print(find_crop_dim(train_image))
print(find_crop_dim(train_label))
print(find_crop_dim(validation_image))
print(find_crop_dim(validation_label))
print(find_crop_dim(test_image))
print(find_crop_dim(test_label))

# the largest possible bounding box is a crop of the smallest square for all images in the entire train, test and validation
dim = min(find_crop_dim(train_image),find_crop_dim(validation_image),find_crop_dim(test_image))
print("the dimension to crop all images to is",dim,"x",dim)

494
494
411
411
572
572
the dimension to crop all images to is 411 x 411


In [7]:
# crop the images using the dimensions found!

def run_crop(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through images in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.bmp'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            crop_to_square(input_path, output_path, dim)

run_crop(train_image,'./data/RoadDetectionFilesCropped/Train/image/')
run_crop(train_label,'./data/RoadDetectionFilesCropped/Train/label/')
run_crop(validation_image,'./data/RoadDetectionFilesCropped/Validation/image/')
run_crop(validation_label,'./data/RoadDetectionFilesCropped/Validation/label/')
run_crop(test_image,'./data/RoadDetectionFilesCropped/Test/image/')
run_crop(test_image,'./data/RoadDetectionFilesCropped/Test/label/')

In [7]:
# save filepaths to cropped folders

train_image_crop = './data/RoadDetectionFilesCropped/Train/image/'
train_label_crop = './data/RoadDetectionFilesCropped/Train/label/'
validation_image_crop = './data/RoadDetectionFilesCropped/Validation/image/'
validation_label_crop = './data/RoadDetectionFilesCropped/Validation/label/'
test_image_crop = './data/RoadDetectionFilesCropped/Test/image/'
test_label_crop = './data/RoadDetectionFilesCropped/Test/label/'

In [8]:
# set order of data layers to match requirements for cnn

from keras import backend as K

if K.image_data_format() == 'channels_first':
    input_shape = (3, dim, dim)
else:
    input_shape = (dim, dim, 3)

In [35]:
# create layers for CNN model process - adapted from a model created in Homework 3

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

m6 = Sequential()
input_shape = (dim, dim, 3)  # Define your image dimensions and number of channels
m6.add(Conv2D(32, (3, 3), input_shape=input_shape))
m6.add(Activation('relu'))
m6.add(MaxPooling2D(pool_size=(2, 2)))

m6.add(Conv2D(32, (3, 3)))
m6.add(Activation('relu'))
m6.add(MaxPooling2D(pool_size=(2, 2)))

m6.add(Conv2D(64, (3, 3)))
m6.add(Activation('relu'))
m6.add(MaxPooling2D(pool_size=(2, 2)))

m6.add(Flatten())
m6.add(Dense(64))
m6.add(Activation('relu'))
m6.add(Dropout(0.5))
m6.add(Dense(1))
m6.add(Activation('sigmoid'))

c:\Users\sofia\mambaforge\envs\musa-550-fall-2023\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [36]:
# this model will be trained for regression because we are trying to predict a singular numeric outcome for each image, for the number of pixels that are roads

# Compile the model for regression
m6.compile(loss='mean_squared_error',
           optimizer='rmsprop',
           metrics=['mean_squared_error']) 


In [33]:
# function to create a dataset that combines the satellite imagery with the number of pixels belonging to roads calculated from the label. Tis will train the model

import numpy as np
import re


def create_model_dataset(label_directory, image_directory):
    # Initialize the dataset
    dataset = []

    # Loop through files in the label directory
    for label_filename in os.listdir(label_directory):
        # Extract numeric part of the filename
        match = re.match(r'(\d+)', label_filename)
        if match:
            base_filename = match.group(1)
        else:
            continue  # Skip this file if no numeric part is found

        # Construct corresponding image filename
        image_filename = 'image' + base_filename + '.bmp'

        # Load label image and calculate number of road pixels
        label_image = Image.open(os.path.join(label_directory, label_filename))
        label_array = np.array(label_image)
        num_road_pixels = np.sum(label_array == 1)

        # Load satellite image
        satellite_image = Image.open(os.path.join(image_directory, image_filename))

        # Append to the dataset
        dataset.append((num_road_pixels, satellite_image))

    # Now you have the dataset ready for training
    return dataset




In [34]:
# create the datasets to pass through the model for training and testing

train_label_crop = './data/RoadDetectionFilesCropped/Train/label'
train_image_crop = './data/RoadDetectionFilesCropped/Train/image'
test_label_crop = './data/RoadDetectionFilesCropped/Test/label'
test_image_crop = './data/RoadDetectionFilesCropped/Test/image'

# Call create_model_dataset function with absolute paths
train_data = create_model_dataset(train_label_crop, train_image_crop)
test_data = create_model_dataset(test_label_crop, test_image_crop)


In [41]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Assuming train_data is your list of tuples containing (num_road_pixels, satellite_image)
train_data = train_data # Your training dataset

# Convert the list of tuples to separate lists for pixels and images
pixels, images = zip(*train_data)
pixels = np.array(pixels)  # Convert to numpy array
images = np.array(images)  # Convert to numpy array

# Create an instance of ImageDataGenerator with any desired preprocessing options
train_datagen = ImageDataGenerator(
    rescale=1. / 255,  # You can add more preprocessing options if needed
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Create a generator for training data
train_generator = train_datagen.flow(
    x=images,
    y=pixels,
    batch_size=32,
    shuffle=True)  # Shuffle the data

# Now you can use train_generator to generate batches of data during model training


ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (c:\Users\sofia\mambaforge\envs\musa-550-fall-2023\lib\site-packages\keras\preprocessing\image\__init__.py)

In [ ]:
# Assuming m6 is your compiled CNN model

# Train the model using fit_generator
history = m6.fit_generator(
    generator=train_generator,
    steps_per_epoch=len(train_data) // 32,
    epochs=10)
